In [2]:
# dei um pip install notify e pip install requests
import requests
from notifypy import Notify

# alertas usando Notify
def alerta(mensagem):  
    notification = Notify()
    notification.title = "Alerta"
    notification.message = mensagem
    notification.send()

In [4]:
# obtendo os dados da API
def fetch_data(url):  
    try:
        response = requests.get(url)
        response.raise_for_status()  # avisar se o status nao for 200
        return response.json()
    except requests.exceptions.RequestException as e:
        alerta(f"Erro ao acessar a API: {e}")
        return None

In [6]:
# URLs da PokeAPI
base_url = "https://pokeapi.co/api/v2/pokemon"

In [29]:
# Pegando os dados das tabelas - Tabela 1: Lista de Pokémon
def get_pokemon_list(limit=20):
    data = fetch_data(f"{base_url}?limit={limit}")
    if data:
        pokemons = [{"id": p["url"].split("/")[-2], "name": p["name"]} for p in data["results"]]
        return pokemons
    return []

In [30]:
# Tabela 2: Tipos de cada Pokémon
def get_pokemon_types(pokemon_name):
    data = fetch_data(f"{base_url}/{pokemon_name}")
    if data:
        types = [t["type"]["name"] for t in data["types"]]
        return {"name": pokemon_name, "types": types}
    return {"name": pokemon_name, "types": []}

In [31]:
# Tabela 3: Estatísticas básicas de cada Pokémon
def get_pokemon_stats(pokemon_name):
    data = fetch_data(f"{base_url}/{pokemon_name}")
    if data:
        stats = {stat["stat"]["name"]: stat["base_stat"] for stat in data["stats"]}
        return {"name": pokemon_name, "stats": stats}
    return {"name": pokemon_name, "stats": {}}

In [32]:
# Testando as funções pra ver se funcionou
pokemon_list = get_pokemon_list(limit=5)

In [33]:
# Colocando as condicionais dos alertas
if not pokemon_list:
    alerta("Nenhum Pokémon encontrado. Verifique a conexão com a API.")
else:
    print("Tabela 1: Lista de Pokémon")
    print(pokemon_list)

    for pokemon in pokemon_list:
        types = get_pokemon_types(pokemon["name"])
        stats = get_pokemon_stats(pokemon["name"])
        if not types["types"]:
            alerta(f"Erro ao buscar tipos do Pokémon {pokemon['name']}.")
        if not stats["stats"]:
            alerta(f"Erro ao buscar estatísticas do Pokémon {pokemon['name']}.")
        
        print(f"Tabela 2: Tipos de {pokemon['name']}")
        print(types)
        print(f"Tabela 3: Estatísticas de {pokemon['name']}")
        print(stats)

Tabela 1: Lista de Pokémon
[{'id': '1', 'name': 'bulbasaur'}, {'id': '2', 'name': 'ivysaur'}, {'id': '3', 'name': 'venusaur'}, {'id': '4', 'name': 'charmander'}, {'id': '5', 'name': 'charmeleon'}]
Tabela 2: Tipos de bulbasaur
{'name': 'bulbasaur', 'types': ['grass', 'poison']}
Tabela 3: Estatísticas de bulbasaur
{'name': 'bulbasaur', 'stats': {'hp': 45, 'attack': 49, 'defense': 49, 'special-attack': 65, 'special-defense': 65, 'speed': 45}}
Tabela 2: Tipos de ivysaur
{'name': 'ivysaur', 'types': ['grass', 'poison']}
Tabela 3: Estatísticas de ivysaur
{'name': 'ivysaur', 'stats': {'hp': 60, 'attack': 62, 'defense': 63, 'special-attack': 80, 'special-defense': 80, 'speed': 60}}
Tabela 2: Tipos de venusaur
{'name': 'venusaur', 'types': ['grass', 'poison']}
Tabela 3: Estatísticas de venusaur
{'name': 'venusaur', 'stats': {'hp': 80, 'attack': 82, 'defense': 83, 'special-attack': 100, 'special-defense': 100, 'speed': 80}}
Tabela 2: Tipos de charmander
{'name': 'charmander', 'types': ['fire']}


In [34]:
# Tratando as Bases
import pandas as pd

In [38]:
# Criando as tabelas com listas e dicionários
pokemon_list = [
    {"id": 1, "name": "bulbasaur"},
    {"id": 2, "name": "ivysaur"},
    {"id": 3, "name": "venusaur"},
]

pokemon_types = [
    {"name": "bulbasaur", "types": ["grass", "poison"]},
    {"name": "ivysaur", "types": ["grass", "poison"]},
    {"name": "venusaur", "types": ["grass", "poison"]},
]

pokemon_stats = [
    {"name": "bulbasaur", "stats": {"hp": 45, "attack": 49, "defense": 49}},
    {"name": "ivysaur", "stats": {"hp": 60, "attack": 62, "defense": 63}},
    {"name": "venusaur", "stats": {"hp": 80, "attack": 82, "defense": 83}},
]

In [40]:
# Convertendo listas de dicionários em DataFrames
df_pokemon = pd.DataFrame(pokemon_list)
df_types = pd.DataFrame(pokemon_types)
df_stats = pd.DataFrame(pokemon_stats)


In [41]:
# Tratamento das Bases

# 1. Ajuste de Nomes das Colunas
df_pokemon.rename(columns={"id": "pokemon_id", "name": "pokemon_name"}, inplace=True)
df_types.rename(columns={"name": "pokemon_name", "types": "pokemon_types"}, inplace=True)
df_stats.rename(columns={"name": "pokemon_name", "stats": "pokemon_stats"}, inplace=True)

In [42]:
# Usando Unstack/Stack para tratamento de Dados - separando os tipos em múltiplas colunas
df_types = df_types.explode("pokemon_types")

In [43]:
# Transformando os stats em colunas
stats_expanded = df_stats["pokemon_stats"].apply(pd.Series)
df_stats = pd.concat([df_stats.drop(columns=["pokemon_stats"]), stats_expanded], axis=1)

In [44]:
# Tratamento de Missing Values
df_pokemon.fillna("Unknown", inplace=True)
df_types.fillna("Unknown", inplace=True)
df_stats.fillna(0, inplace=True)

In [45]:
# Tratamento de Tipos de Variáveis
df_stats = df_stats.astype({"hp": int, "attack": int, "defense": int})

In [49]:
# Filtragem de Dados - selecionando apenas Pokémon com hp > 10
df_stats_filtered = df_stats[df_stats["hp"] > 50]

In [50]:
# Salvando as Bases Tratadas
df_pokemon.to_csv("pokemon_list.csv", index=False)
df_types.to_csv("pokemon_types.csv", index=False)
df_stats_filtered.to_csv("pokemon_stats_filtered.csv", index=False)

In [52]:
# Verificando as tabelas tratadas
print("Tabela Pokémon:")
print(df_pokemon.head())
print("\nTabela Tipos:")
print(df_types.head())
print("\nTabela Estatísticas Filtradas:")
print(df_stats_filtered.head())

Tabela Pokémon:
   pokemon_id pokemon_name
0           1    bulbasaur
1           2      ivysaur
2           3     venusaur

Tabela Tipos:
  pokemon_name pokemon_types
0    bulbasaur         grass
0    bulbasaur        poison
1      ivysaur         grass
1      ivysaur        poison
2     venusaur         grass

Tabela Estatísticas Filtradas:
  pokemon_name  hp  attack  defense
1      ivysaur  60      62       63
2     venusaur  80      82       83
